# Application of Fuzzy logic and Genetic Algorithm in Stock Trading

This notebook explores the application of genetic algorithms in optimizing the fuzzy rules as applied in technical analysis of stocks.

In [ ]:
# import libraries
import warnings
warnings.filterwarnings('ignore')
import math
import random
import multiprocessing
from functools import partial
import itertools
from typing import Union
from re import X
import pandas as pd
import matplotlib.pyplot as plt
import copy
from fuzzy_ta import fuzzy_TA
from Gene import Gene
from Genome import Genome
from Population import Population
from Crossover import single_point, two_point, uniform, linear, SBX
from Fitness import evaluate_fitness
from Selection import roulette_wheel

In [ ]:
# read and print sample historical stock price data
series = pd.read_csv('../../Data/PH-historical-stock-price-data-csv/GLO.csv', header = 0, index_col = 'Date')
print(series)

# plot historical stock price data
fig, ax = plt.subplots()    
ax.set_title('GLO closing prices')
ax.set_xlabel('Date')
ax.set_ylabel('Price (Pesos)')
ax.plot(series['Close'].tail(60))
ax.set_xticks(ax.get_xticks()[::13])
plt.show()

In [ ]:
# create a function that splits the data into train and test sets
def split_train_test_sets(series: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    This function splits the time series data into train and test sets
    
    Arguments:
        series: pd.DataFrame
            a time series data of type
            
    Returns:
        train, test: tuple(pd.DataFrame, pd.DataFrame)
            the splitted train and test sets
    """
    # split into train and test sets
    train, test = series[1:-math.floor(len(series)*0.2)], series[-math.floor(len(series)*0.2):]
    return train, test

In [ ]:
# split data into train and test sets
train, test = split_train_test_sets(series)

# plot the train and test sets
fig, ax = plt.subplots()    
ax.set_title('GLO closing prices')
ax.set_xlabel('Date')
ax.set_ylabel('Price (Pesos)')
ax.plot(train['Close'], 'g')
ax.plot(test['Close'], 'r')
ax.set_xticks(ax.get_xticks()[::2000])
plt.show()

In [ ]:
def split_train_set(series:pd.DataFrame, window:int = 500) -> list[pd.DataFrame]:
    """
    This function splits or slices the train set into batches with a corresponding window length

    Arguments:
        series:pd.DataFrame
            a pandas dataframe containing the training set of the 

    Returns:
        train_set:list[pd.DataFrame]
            a list containing multiple pandas dataframes
    """
    train_set = list()
    for i in range(len(series)-window):
        train_set.append(copy.deepcopy(series[i:i+window]))
    return train_set

In [ ]:
# split the train set into multiple batches
train_set = split_train_set(train)
print(len(train_set))

In [ ]:
# create the gene list
gene_list = list()
gene_list = [
                Gene(
                    name = "RSI_window",
                    lower_bound = 1,
                    upper_bound = 300,
                    type = "int"),
            
                Gene(
                    name = "RSI_p1",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float"),
                
                Gene(
                    name = "RSI_p2",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float"),
                
                Gene(
                    name = "RSI_p3",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float"),

                Gene(
                    name = "RSI_p4",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float"),

                Gene(
                    name = "RSI_low_membership",
                    lower_bound = 0,
                    upper_bound = 100,
                    type = "linear_membership"),

                Gene(
                    name = "RSI_middle_membership",
                    lower_bound = 0,
                    upper_bound = 100,
                    type = "triangular_membership"),

                Gene(
                    name = "RSI_high_membership",
                    lower_bound = 0,
                    upper_bound = 100,
                    type = "linear_membership"),

                Gene(
                    name = "entry_condition",
                    lower_bound = 1,
                    upper_bound = 100,
                    type = "entry_condition"),

                Gene(
                    name = "stop_loss",
                    lower_bound = 0.01,
                    upper_bound = 0.99,
                    type = "float"),

                Gene(
                    name = "z_rolling_window",
                    lower_bound = 1,
                    upper_bound = 300,
                    type = "int")

            ]

In [ ]:
# create the seed gene; this gene contains initial values believed to be good instance values
seed_gene_list = list()
seed_gene_list = [
                Gene(
                    name = "RSI_window",
                    lower_bound = 1,
                    upper_bound = 300,
                    type = "int",
                    value = 30),
            
                Gene(
                    name = "RSI_p1",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float",
                    value = 1),
                
                Gene(
                    name = "RSI_p2",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float",
                    value = 1),
                
                Gene(
                    name = "RSI_p3",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float",
                    value = 1),

                Gene(
                    name = "RSI_p4",
                    lower_bound = -1,
                    upper_bound = 1,
                    type = "float",
                    value = 1),

                Gene(
                    name = "RSI_low_membership",
                    lower_bound = 0,
                    upper_bound = 100,
                    type = "linear_membership",
                    value = [0, 25]),

                Gene(
                    name = "RSI_middle_membership",
                    lower_bound = 0,
                    upper_bound = 100,
                    type = "triangular_membership",
                    value = [25, 50, 75]),

                Gene(
                    name = "RSI_high_membership",
                    lower_bound = 0,
                    upper_bound = 100,
                    type = "linear_membership",
                    value = [75,100]),

                Gene(
                    name = "entry_condition",
                    lower_bound = 1,
                    upper_bound = 100,
                    type = "entry_condition",
                    value = [50,50]),

                Gene(
                    name = "stop_loss",
                    lower_bound = 0.01,
                    upper_bound = 0.99,
                    type = "float",
                    value = 0.95),

                Gene(
                    name = "z_rolling_window",
                    lower_bound = 1,
                    upper_bound = 300,
                    type = "int",
                    value = 30)

            ]

In [ ]:
genome1 = Genome(seed_gene_list)
print(genome1)

In [ ]:
# genome1 = Genome(seed_gene_list)
genome1 = Genome(gene_list)
genome1.initialize_genome()
# stock, num_trades, bnh_returns, strat_returns, strat_sharpe_ratio, strat_sortino_ratio, max_drawdown= evaluate_fitness(train_set[2500], genome1)
strat_sortino_ratio = evaluate_fitness(train_set[2500], genome1)
# print(returns)
# print(num_trades)
# print(bnh_returns)
# print(strat_returns)
# print(strat_sharpe_ratio)
print(strat_sortino_ratio)
# print(max_drawdown)

In [ ]:
genome1 = Genome(seed_gene_list)
genome2 = Genome(gene_list)
population1 = Population()
population1.seed_population(seed_genome = genome1, num_seeds = 1)
population1.initialize_population(num_genomes = 2, genome = genome2)
for genome in population1.population:
    print(genome)
    

In [ ]:
genome1 = Genome(seed_gene_list)
genome2 = Genome(gene_list)
genome2.initialize_genome()

offspring1, offspring2 = SBX(genome1, genome2)
print(offspring1)
# print(offspring2)


In [ ]:
class Evolution():
    """
    This class provides details on the evolution class
    """
    
    def __init__(self, population:Population):
        """
        This function initializes the evolution class
        """
        self.population = population
    
    def run(self, fitness:callable, generations:int = 100, checkpoint_interval:int = 10):
        """
        This function simulates evolution through the population
        
        Arguments:            
            generations: int
                the number of generations the evolution should run
                
            checkpoint_interval: int
                the interval for saving a checkpoint in the evolution of the genomes
                
        Returns:
            None
        """
        for _ in range(generations):
            pass
    
    def checkpoint():
        """
        Some text
        """
        pass
                

In [ ]:
genome1 = Genome(seed_gene_list)
genome2 = Genome(gene_list)
population1 = Population()
population1.seed_population(seed_genome = genome1, num_seeds = 25)
population1.initialize_population(genome = genome2, num_genomes = 75)
# evolution1 = Evolution(population1)
# for genome in population1.population:
#     print(genome)

In [ ]:
# evaluate_fitness_partial = partial(evaluate_fitness, series = train_set[2500])
genome1 = Genome(seed_gene_list)
genome2 = Genome(gene_list)
population1 = Population()
population1.seed_population(seed_genome = genome1, num_seeds = 25)
population1.initialize_population(genome = genome2, num_genomes = 75)

roulette_wheel(population = population1, fitness_func = evaluate_fitness, series = train_set[2500])
# roulette_wheel(population = population1, fitness_func = evaluate_fitness_partial, series = train_set[2500])

In [ ]:
def run_evolution(population = None, generations:int = None, checkpoint_interval:int = None) -> None:
    """
    This function simulates evolution through the population
    of genes
    
    Arguments:
        population: population class
            a class of population
        
        generations: int
            the number of generations the evolution should run
            
        checkpoints: int
            the interval for saving a checkpoint in the evolution of the genomes
            
    Returns:
        None
        
    """
    pass

    # TODO: include checkpoints here




In [ ]:
def main() -> None:
    """
    This function runs all the other functions as provided in the above
    """
    pass
    # use multiprocessing here

In [ ]:
# TODO: improve this notebook by using NSGA III in future implementations.
# TODO: visualize the fitness of the generation through PCA
# TODO: refactor SBX crossover